<a href="https://colab.research.google.com/github/Meta-Sean/Practical-Deep-Learning/blob/main/corncomp_Kfolds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 26 14:51:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P0    38W / 250W |   4031MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [43]:
!pip install kaggle fastkaggle fastai timm>=0.6.2.dev0 pynvml

In [44]:
from google.colab import files
import pandas as pd
uploaded = files.upload()

Saving kaggle.json to kaggle (1).json


In [45]:
! mkdir ~/.kaggle #creating folder
! cp kaggle.json ~/.kaggle/ #copying kaggle.json
! chmod 600 ~/.kaggle/kaggle.json #reading the file with full access

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [46]:
from fastkaggle import * 
from fastai.vision.all import *
import numpy as np
from sklearn.model_selection import KFold

comp = 'kaggle-pog-series-s01e03'

path = setup_comp(comp, install='"fastcore>=1.4.5" "fastai>=2.7.1" "timm>=0.6.2.dev0"')

In [47]:
from pathlib import Path
path = Path('/content/kaggle-pog-series-s01e03/corn/')
path

Path('/content/kaggle-pog-series-s01e03/corn')

In [48]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [49]:
trn_path = path/'train'
files = get_image_files(trn_path)
tst_files = get_image_files(path/'test').sorted()

In [50]:
train_df = pd.read_csv(path/'train.csv')
train_df.head()

,seed_id,view,image,label
0,0,top,train/00000.png,broken
1,1,bottom,train/00001.png,pure
2,3,top,train/00003.png,broken
3,4,top,train/00004.png,pure
4,5,top,train/00005.png,discolored


In [69]:
df = pd.read_csv(path/'train.csv')
train_df.label.value_counts() * 100 / len(train_df)
train_df.label.unique()

array(['broken', 'pure', 'discolored', 'silkcut'], dtype=object)

In [70]:
train_df['kfold'] = -1

# Number of Splits
n_folds=5
reversed = False

# For each label we are going to create n_folds folds
for label in train_df.label.unique():
    
    # Assign fold number from 0 to n_folds or from n_folds to 0
    # Because KFold assigns less data for last fold (we assign it to fold 0 or n_folds)
    folds = list(range(n_folds))
    if reversed: folds.reverse()
    
    kf = KFold(n_splits=n_folds, random_state=42, shuffle=True)
    
    # Indices for each label
    label_idxs = train_df[train_df.label==label].index
    
    # Creating folds for those indices
    kf.get_n_splits(label_idxs)

    for _, valid_index in kf.split(label_idxs):

        actual_fold = folds.pop(0)
        df_index = label_idxs[valid_index]
        train_df.loc[df_index, 'kfold'] = actual_fold
    reversed = not reversed
        

In [71]:
img2valid = []

for fold in range(n_folds):
    train_df['is_valid'] = False
    idxs = train_df[train_df.kfold == fold].index
    train_df.loc[idxs, 'is_valid'] = True
    
    img2valid.append({ r.image: r.is_valid for _, r in train_df.iterrows() })

In [95]:
temp_df = train_df.set_index('image')
temp_df
def y_label(o):
  temp = str(o)
  return temp_df.loc[temp[-15:]]['label']

temp_df.loc['train/01778.png']['label']

'pure'

In [96]:
def get_datablock(i_fold, size, item_tfms, accum):
    
    def get_split(p):
        # For each fold, return if an image is in valid set or not
        return img2valid[i_fold]['train/'+p.name]
    
    dblock = DataBlock(
        blocks=(ImageBlock, CategoryBlock),
        get_items=get_image_files,
        get_y=y_label,
        # Custom Splitter
        splitter = FuncSplitter(get_split),
        item_tfms=item_tfms,
        batch_tfms=aug_transforms(size=size, min_scale=0.75)
    )
    return dblock.dataloaders(trn_path, bs=64//accum)

In [97]:
def train(i_fold, arch, size=132, item_tfms=Resize(132, method='squish'), accum=1, epochs=5, lr=0.01):
    
    dls = get_datablock(i_fold=i_fold, size=size, item_tfms=item_tfms, accum=accum)
    print('- First 5 validation images:')
    print([each.name for each in dls.valid.items[:5]])
    
    cbs = GradientAccumulation(64) if accum!=1 else []
    
    # Force torchvision models instead of TIMM, when possible
    try: arch = eval(arch)
    except: arch = arch
        
    learn = vision_learner(dls, arch, metrics=accuracy, cbs=cbs).to_fp16()
    print('- Fine Tuning')
    learn.fine_tune(epochs, lr)
    #print('- Getting predictions')
    #probs, _ = learn.get_preds(dl=dls.test_dl(test_files))
    probs = None # Return only tta_preds
    print('- Getting tta_predictions')
    preds, _ = learn.tta(dl=dls.test_dl(tst_files))
    
    return probs, preds, dls.vocab


In [98]:
models = {
    'convnext_small_in22k': {
        (0, Resize(132, method='squish'), 132),
        (1, Resize(132, method='squish'), 132),
        (2, Resize(132, method='squish'), 132),
        (3, Resize(132, method='squish'), 132),
        (4, Resize(132, method='squish'), 132)
    }
    
}

In [99]:
predictions = []
tta_predictions = []

In [100]:
ss = pd.read_csv(path/'sample_submission.csv')
ss

,seed_id,label
0,8632,broken
1,11394,broken
2,17362,pure
3,9987,discolored
4,17226,silkcut
...,...,...
3474,1461,pure
3475,2566,discolored
3476,11504,discolored
3477,5140,pure


In [ ]:

exp = 1
for arch, details in models.items():
    
    for i_fold, item, size in details:
        print('////'*10)
        print('---Experiment', exp, '--', arch)
        print('fold: ', i_fold)
        print(item.name)
        
        preds, tta_preds, vocab = train(i_fold, arch, size, item_tfms=item, accum=1, epochs=5, lr=0.01)
        
        predictions.append(preds)
        tta_predictions.append(tta_preds)
        
        now = datetime.now().strftime("%Y%m%d")
        filename = f'{now}-exp{exp}-{arch}-Fold{i_fold}.csv'
        print(f'Saving {filename}')
        
        ss.label = vocab[tta_preds.argmax(axis=1)]
        ss.to_csv(filename, index=False)
        
        gc.collect()
        torch.cuda.empty_cache()
        exp += 1

////////////////////////////////////////
---Experiment 1 -- convnext_small_in22k
fold:  0
Resize -- {'size': (132, 132), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}
- First 5 validation images:
['11464.png', '08893.png', '16825.png', '06011.png', '03216.png']
- Fine Tuning


epoch,train_loss,valid_loss,accuracy,time
0,1.036533,0.780122,0.709846,01:00


epoch,train_loss,valid_loss,accuracy,time
0,0.709652,0.621045,0.750000,01:45
1,0.649077,0.623016,0.768156,01:46
2,0.527368,0.546365,0.780726,01:46
3,0.435984,0.509842,0.802723,01:45
4,0.366133,0.520270,0.799581,01:45


- Getting tta_predictions


Saving 20220926-exp1-convnext_small_in22k-Fold0.csv
////////////////////////////////////////
---Experiment 2 -- convnext_small_in22k
fold:  1
Resize -- {'size': (132, 132), 'method': 'squish', 'pad_mode': 'reflection', 'resamples': (2, 0), 'p': 1.0}
- First 5 validation images:
['04415.png', '03242.png', '16100.png', '11993.png', '09672.png']
- Fine Tuning


epoch,train_loss,valid_loss,accuracy,time


In [86]:
[each.shape for each in tta_predictions]

[]

In [ ]:
avg_tta_predictions = torch.stack(tta_predictions).mean(0)
avg_tta_predictions.shape

In [ ]:
vocab[avg_tta_predictions.argmax(dim=1)]

In [ ]:
save_pickle('tta_res.pkl', tta_res)

In [ ]:
tta_prs = first(zip(*tta_res))

In [ ]:
tta_res

In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

In [ ]:
dls = ImageDataLoaders.from_csv(path, 'train.csv', fn_col=2, label_col=3, item_tfms=Resize(132, method='squish'),
                                   batch_tfms=aug_transforms(size=128, min_scale=0.75))

In [ ]:
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)


In [ ]:
avg_pr

In [ ]:
ss = pd.read_csv(path/'sample_submission.csv')
ss = ss['seed_id']
tst_df = pd.read_csv(path/'test.csv')
tst_df
ss

In [ ]:
tst_df['label'] = vocab[idxs]
tst_df

In [ ]:
ss = pd.merge(ss, tst_df, on="seed_id")
ss

In [ ]:
ss = ss[["seed_id", "label"]]
ss

In [ ]:
ss.to_csv('submission.csv', index=False)
!head submission.csv